In [ ]:
import numpy as np
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from lac.slam.semantic_feature_tracker import SemanticFeatureTracker
from lac.perception.segmentation import SemanticClasses
from lac.slam.frontend import Frontend
from lac.slam.backend import Backend
from lac.utils.plotting import plot_poses, plot_loop_closures, plot_3d_points, plot_semantic_points
from lac.util import load_data, load_stereo_images, load_images, positions_rmse_from_poses
from lac.utils.geometry import crop_points
from lac.params import SCENE_BBOX

%load_ext autoreload
%autoreload 2

# Load data


In [ ]:
# Load the data logs
data_path = "/home/shared/data_raw/LAC/runs/double_loop_preset1"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
print(f"Loaded {len(poses)} poses")

In [ ]:
# images = load_images(data_path, cameras=["FrontLeft", "FrontRight"], start_frame=0, end_frame=10000)
left_imgs, right_imgs = load_stereo_images(data_path, start_frame=0)
images = {"FrontLeft": left_imgs, "FrontRight": right_imgs}

# Run slam for local map


In [ ]:
START_FRAME = 80
END_FRAME = 200

# TODO: we should probably keyframe the first frame (give it to the backend)

feature_tracker = SemanticFeatureTracker(cam_config)
frontend = Frontend(feature_tracker)
backend = Backend(poses[START_FRAME], feature_tracker)

frontend.initialize(left_imgs[START_FRAME], right_imgs[START_FRAME])

In [ ]:
for frame in tqdm(range(START_FRAME + 2, END_FRAME, 2)):
    data = {
        "step": frame,
        "FrontLeft": left_imgs[frame],
        "FrontRight": right_imgs[frame],
        "imu": imu_data[frame],
    }
    data = frontend.process_frame(data)
    backend.update(data)

In [ ]:
points = frontend.feature_tracker.tracked_points.points_local
labels = frontend.feature_tracker.tracked_points.labels
plot_semantic_points(
    points, labels, names=[member.name for member in SemanticClasses], markersize=2
)